### **QLoRA**

In [ ]:
pip install bitsandbytes git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/accelerate.git datasets trl einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-y91zuhjw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-y91zuhjw
  Resolved https://github.com/huggingface/transformers.git to commit 70c79940957fb25b54bd1b106935c756b90345eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-uh2ct8c2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-uh2ct8c2
  Resolved https://github.com/huggingface/peft.git to commit 189a6b8e357ecda05ccde13999e4c35759596a67
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... 

In [ ]:
!pip install git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers (to revision de9255de27abfcae4a1f816b904915f0b1e23cd9) to /tmp/pip-req-build-ytcxfn0o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ytcxfn0o
  Running command git rev-parse -q --verify 'sha^de9255de27abfcae4a1f816b904915f0b1e23cd9'
  Running command git fetch -q https://github.com/huggingface/transformers de9255de27abfcae4a1f816b904915f0b1e23cd9
  Running command git checkout -q de9255de27abfcae4a1f816b904915f0b1e23cd9
  Resolved https://github.com/huggingface/transformers to commit de9255de27abfcae4a1f816b904915f0b1e23cd9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.30.0.dev0-py3-none-any.whl size=7128533 sha256=88a64a

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch, einops
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments
)
from peft.tuners.lora import LoraLayer

from trl import SFTTrainer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
def create_and_prepare_model():
    compute_dtype = getattr(torch, "float16")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        "tiiuae/falcon-7b", quantization_config=bnb_config, device_map={"": 0}, trust_remote_code=True
    )

    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "query_key_value"
        ],
    )

    tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b", trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    return model, peft_config, tokenizer

In [ ]:
def concat(x):
    concated = ""
    for k, v in x.items():
        if k == "spoiler":
            continue
        concated += k + ": " + str(v) + "\n"
    concated += "spoiler" + ": " + x["spoiler"] + "\n"
    return concated

def concat_val(x):
    concated = ""
    for k, v in x.items():
        if k == "spoiler" or k=="provenance":
            continue
        concated += k + ": " + str(v) + "\n"
    concated += "spoiler" + ": "
    return concated

In [ ]:
data = load_dataset("json", data_files="/content/drive/MyDrive/NLP_Project/train_changed.json", split="train")

In [ ]:
dataset = data.map(lambda x: {"text": concat(x)})

In [ ]:
dataset

Dataset({
    features: ['postText', 'postPlatform', 'targetParagraphs', 'targetTitle', 'targetDescription', 'targetKeywords', 'provenance', 'spoiler', 'spoilerPositions', 'tags', 'text'],
    num_rows: 3197
})

In [ ]:
model, peft_config, tokenizer = create_and_prepare_model()
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
training_arguments = TrainingArguments(
    output_dir="/content/falcon",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=10,#max_steps=10000,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/3197 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.136500
20,2.089500
30,2.137900
40,2.099300
50,2.093800
60,2.018900
70,2.124600
80,2.102200
90,2.079100
100,2.022700


TrainOutput(global_step=430, training_loss=1.9660607493200968, metrics={'train_runtime': 2403.4628, 'train_samples_per_second': 0.728, 'train_steps_per_second': 0.179, 'total_flos': 7.042619286749184e+16, 'train_loss': 1.9660607493200968, 'epoch': 9.83})

In [ ]:
trainer.save_model('last-2048-10')

In [ ]:
def tokenization(example):
    return len(tokenizer(example)['input_ids'])

In [ ]:
import numpy as np
ddd = [tokenization(x) for x in dataset['text']]

In [ ]:
np.sum(np.unique(ddd, return_counts=True)[1][np.unique(ddd, return_counts=True)[0] > 2048])

175

### Validation Data

In [ ]:
valdata = load_dataset("json", data_files="/content/drive/MyDrive/NLP_Project/validation_changed.json", split="train")

In [ ]:
valdataset = valdata.map(lambda x: {"text": concat_val(x)})

In [ ]:
valdataset['text'][1]

'postText: Why Arizona Sheriff Joe Arpaio’s fate could hang on a single word\npostPlatform: reddit\ntargetParagraphs: © REUTERS/Laura Segall Maricopa County Sheriff Joe Arpaio announces newly launched program aimed at providing security around schools in Anthem, Arizona, January 9, 2013. PHOENIX — A single word — "intentionally" — could transform a court case against Phoenix-area Sheriff Joe Arpaio from civil charges to a criminal prosecution.??? In finding Arpaio and three of his top commanders in contempt of court on Friday, U.S. District Court Judge Murray Snow said no fewer than 19 times that the sheriff or his aides had repeatedly and intentionally violated judicial orders to stop profiling Latinos. On May 31, Snow will determine the civil penalties and examine whether Arpaio, the sheriff of Arizona’s Maricopa County, will be referred to Arizona’s U.S. attorney for potential criminal charges.??? Arpaio has acknowledged violating Snow’s order, but said it was a mistake. In weighing

In [ ]:
model.eval()

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear4bit(
            in_features=4544, out_features=4672, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4544, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4672, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (dense): Linear4bit(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0,

In [ ]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusFor

In [ ]:
def save_nested_strings_to_file(nested_strings, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for inner_list in nested_strings:
            file.write('$$$\n')
            for string in inner_list:
                file.write(string + '\n')
            file.write('$$$')
            file.write('\n')

In [ ]:
sequences = pipeline(
    valdataset['text'][3],
    max_new_tokens=128,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Token indices sequence length is longer than the specified maximum sequence length for this model (2887 > 2048). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: postText: "Harry Potter" alums reunite for new movie
postPlatform: Twitter
targetParagraphs: The mythology of punk music's evolution can be traced back, more or less, to one singular staple: CBGB. Opened in 1973 in downtown Manhattan's East Village, the nightclub that originally intended to house country, bluegrass and blues music (hint its acronym) quickly became a breeding ground for bands like The Ramones, Talking Heads and The Police, among countless others.??? The legacy of the club, which closed in 2006 over a rent dispute, is on display in a new movie that stars Alan Rickman as bemused owner Hilly Kristal. It's a bit of a departure for anyone more accustomed to seeing Rickman, 67, play sinister Severus Snape in "Harry Potter" or Judge Turpin in "Sweeney Todd." The movie also afforded him the opportunity to reunite with fellow Hogwarts alum Rupert Grint, who portrays one-third of derriere-bearing punk band The Dead Boys, alongside Justin Bartha and Bronson Adams. They're 

In [ ]:
from tqdm import tqdm

save_list =[]
for text in tqdm(valdataset['text']):
    sequences = pipeline(
        text,
        max_new_tokens=128,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    output = []
    for seq in sequences:
        output.append(seq['generated_text'])
        #print(f"Result: {seq['generated_text']}")
    save_list.append(output)
    torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
save_nested_strings_to_file(save_list, "/content/drive/MyDrive/NLP_Project/falcon/val_2048_10_200.txt")